# Import modules

In [2]:
#importing modules
import numpy as np
import pandas as pd 
import scipy
from scipy import stats
from scipy.stats import norm
import csv

In [3]:
#import the data files as dataframes in pandas
mags_optim = pd.read_csv('ANNZ_randomReg_0000_mags_optim_eval.csv').reset_index().values
mags_eval = pd.read_csv('ANNZ_randomReg_0000_mags_eval.csv').reset_index().values
colors_optim = pd.read_csv('ANNZ_randomReg_0000_colors_optim_eval.csv').reset_index().values
colors_eval = pd.read_csv('ANNZ_randomReg_0000_colors_eval.csv').reset_index().values

In [ ]:
file_type = {'mags_optim':mags_optim, 'mags_eval':mags_eval, 'colors_optim':colors_optim, 'colors_eval':colors_eval} #create a dictionary holding the file arrays
ztype = globals() #to hold the variables related to z
pdf_sets = globals() #to hold the arrays for each PDF
pdf_size = np.linspace(0.0,1.0,200) #the number of points in each PDF, same for both sets, and was set to 200 intervals

In [ ]:
for k, v in file_type.items(): #iterate through the keys and their respective values(arrays) in the file_type dictionary
    ztype['{}_range'.format(k)] = len(v) #find the length of each array
    if 'optim' in k: #run for optimisation files, need to differentiate due to slightly different positioning of values in optimisation and evaluation csv files
        for l in range (0, ztype['{}_range'.format(k)]):
            pdf_sets['{}_row{}'.format(k, l)] = v[[l], None, :]
            pdf_sets['{}_row{}_reshape'.format(k, l)] = np.reshape(pdf_sets['{}_row{}'.format(k, l)], (417,1))
            pdf_sets['{}_row{}_zpdf0'.format(k, l)] = pdf_sets['{}_row{}_reshape'.format(k, l)][11:211]
            pdf_sets['{}_row{}_zpdf1'.format(k, l)] = pdf_sets['{}_row{}_reshape'.format(k, l)][217:417]
    elif 'eval' in k: #run for evaluation csvs
        for l in range (0, ztype['{}_range'.format(k)]):
            pdf_sets['{}_row{}'.format(k, l)] = v[[l], None, :]
            pdf_sets['{}_row{}_reshape'.format(k, l)] = np.reshape(pdf_sets['{}_row{}'.format(k, l)], (420,1))
            pdf_sets['{}_row{}_zpdf0'.format(k, l)] = pdf_sets['{}_row{}_reshape'.format(k, l)][14:214]
            pdf_sets['{}_row{}_zpdf1'.format(k, l)] = pdf_sets['{}_row{}_reshape'.format(k, l)][220:420]
    else:
        print('File type not recognised, please include the strings "optim" or "eval" in file name, depending on file type')

In [ ]:
for k in file_type.keys():
    for r in range(0, 2):
        pdf_names['{}_zpdf{}_sample_array'.format(k, r)] = np.zeros([1,1])
        for l in range (0, ztype['{}_range'.format(k)]): #run for the length of the dataset
            pdf_names['{}_row{}_zpdf{}_1D'.format(k, l, r)] = np.reshape(pdf_names['{}_row{}_zpdf{}'.format(k, l, r)], len(pdf_names['{}_row{}_pdf{}'.format(k, l, r)])) #reshape each PDF array from a 2D into a 1D array, required in order to use np.random.choice 
            pdf_names['{}_row{}_zpdf{}_1D'.format(k, l, r)] /= pdf_names['{}_row{}_zpdf{}_1D'.format(k, l, r)].sum() #normalise the PDF sums to equal 1, as the p variable in np.random.choice needs to be equal to 1, and the sums are not precisely equal to 1 (but close)
            pdf_names['{}_row{}_zpdf{}_sample'.format(k, l, r)] = np.random.choice(pdf_array_size, p=pdf_names['mags_o_e_row_%d_pdf_0_1D' % v]) #create a z value for each entry, using the PDF values for each entry to represent the probabilities  
            pdf_names['{}_zpdf{}_sample_array'.format(k, r)] = np.append(pdf_names['{}_zpdf{}_sample_array'.format(k, r)], pdf_names['{}_row{}_zpdf{}_sample'.format(k, l, r)]) #append the values onto the array 

        pdf_names['{}_zpdf{}_sample_array'.format(k, r)] = pdf_names['{}_zpdf{}_sample_array'.format(k, r)][1:]

In [ ]:
for k in file_type.keys():
    for r in range(0, 2):
        pdf_names['{}_zpdf{}_sample_array'.format(k, r)].tofile('{}_zpdf{}_sample_array.csv'.format(k, r),sep=',',format='%10.5f')